# Code for Lecture-3 of Short Course of Temporal Point Processes

In [88]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F

In [89]:
# fix random seed 
np.random.seed(12345)
torch.random.manual_seed(12345)

## Neural Hawkes Process

### Continuous-time LSTM cell

The LSTM cell $c(t)$ drifts from $c_{\text{start}}$ towards $c_{\text{target}}$. 

In [90]:
class CTLSTMCell(nn.Module): 
    
    def __init__(self, hdim): 
        super(CTLSTMCell, self).__init__()
        """
        hdim : # of hidden neurons
        """
        self.hdim = hdim 
        self.lin = nn.Linear(hdim*2, hdim*6, bias=True)
        return 
    
    def forward(self, x, h, c, ct): 
        """
        x : input embedding
        h : hidden state right before time t 
        c : LSTM cell right before time t 
        ct : LSTM target cell given current history
        """
        x = torch.cat((x, h), dim=0)
        y = self.lin(x)
        
        gi, gf, z, git, gft, gd = y.chunk(6, 0)
        
        gi = torch.sigmoid(gi)
        gf = torch.sigmoid(gf)
        z = torch.tanh(z)
        git = torch.sigmoid(git)
        gft = torch.sigmoid(gft)
        gd = F.softplus(gd)
        
        cs = gf * c + gi * z 
        ct = gft * ct + git * z
        
        return cs, ct, gd
    
    def decay(self, cs, ct, gd, dt): 
        """
        cs : LSTM start cell
        ct : LSTM target cell 
        gd : decay gate
        dt : elapsed time 
        """
        c = ct + (cs - ct) * torch.exp(-gd * dt)
        h = torch.tanh(c)
        
        return c, h

### Neural Hawkes process

The intensity is defined as $\lambda_k(t) = \text{Softplus}(\text{Linear}(h(t)))$.  

In [91]:
class NHP(nn.Module): 
    
    def __init__(self, kdim, hdim): 
        super(NHP, self).__init__()
        """
        kdim : # of event types 
        hdim : # of hidden neurons
        """
        self.eps = np.finfo(float).eps 
        self.max = np.finfo(float).max 
        self.kdim = kdim 
        self.hdim = hdim 
        self.BOS = kdim 
        
        self.emb_in = nn.Embedding(kdim+1, hdim)
        self.ctlstm = CTLSTMCell(hdim)
        self.emb_out = nn.Linear(hdim, kdim)
        
        self.cs = torch.zeros(size=[hdim], dtype=torch.float32)
        self.ct = torch.zeros(size=[hdim], dtype=torch.float32)
        self.gd = torch.zeros(size=[hdim], dtype=torch.float32)
        
        return 
    
    def start(self): 
        self.cs = torch.zeros(size=[hdim], dtype=torch.float32)
        self.ct = torch.zeros(size=[hdim], dtype=torch.float32)
        self.gd = torch.zeros(size=[hdim], dtype=torch.float32)
        self.update(self.BOS, 0.0)
        return 
    
    def update(self, k, dt): 
        """
        k : event type 
        dt : elapsed time since last event
        """
        c, h = self.ctlstm.decay(self.cs, self.ct, self.gd, dt)
        x = self.emb_in(torch.LongTensor([k]))[0]
        self.cs, self.ct, self.gd = self.ctlstm(x, h, c, self.ct)
        
        return 
    
    def forward(self, k, dt): 
        self.update(k, dt)
        return 
    
    def compute_intensities(self, dt): 
        c, h = self.ctlstm.decay(self.cs, self.ct, self.gd, dt)
        return F.softplus(self.emb_out(h))
    
    def compute_total_intensity(self, dt): 
        intensities = self.compute_intensities(dt)
        return torch.sum(intensities)

### Draw a sequence of events by thinning algorithm

For the code to be easy to understand, I only have non-vectorized implementation. Please check the repos for my published papers for highly vectorized and optimized implementation. 

In [92]:
def thinning(model): 
    dt = 0.0
    bound = 100.0 
    # manualy chosen for simplicity
    # in principle, it can be found using the method in Appendix B.3 of Mei & Eisner 2017
    while True: 
        u = np.random.uniform(0.0, 1.0)
        dt += -np.log(1-u) / bound
        intens = model.compute_intensities(dt)
        total_inten = torch.sum(intens)
        accept_prob = total_inten / bound
        u = np.random.uniform(0.0, 1.0)
        if u <= accept_prob: 
            break 
    
    k = torch.multinomial(intens, 1)
    
    return k, dt
    

Draw data from a low-entropy distribution: (1) draw $dt$ from a univariate NHP; (2) draw $k$ from a n-gram model.

In [101]:
kdim = 32 
hdim = 8
nhp = NHP(1, hdim)
# init by BOS 
nhp.start()

T = 100.0
t = 0
seq = []
CONTEXT = 0

while True:
    # draw dt using thinning algorithm
    
    _, dt = thinning(nhp)
    t += dt
    if t <= T: 
        k = (CONTEXT + 1) % kdim # increase event type ID by +1 mod K
        seq += [(dt, k)] # track dt, not t, easy to use
        # update model 
        nhp.forward(0, dt)
        # update CONTEXT 
        CONTEXT = k
    else: 
        break

print(f"over time interval [0, {T}]")
print(f"# of events : {len(seq)}")

over time interval [0, 100.0]
# of events : 88


### Train NHP by MLE

In [102]:
def mle(data, model): 
    """
    compute log-likelihood of seq under model
    """
    J = 10
    model.start()
    loglik = 0.0
    
    """
    sum log intensity - integral total intensity
    """
    
    for event in seq: 
        dt, k = event
        # log intensity 
        loglik += torch.log(model.compute_intensities(dt)[k])
        # integral
        integral = 0.0 
        for j in range(J): 
            # draw uniform-distributed time points
            dtj = np.random.uniform(0.0, dt)
            integral += model.compute_total_intensity(dtj)
        integral /= J 
        integral *= dt 
        loglik -= integral
        # update model
        model.forward(k, dt)
    
    return loglik

In [103]:
import time
import torch.optim as optim

model = NHP(kdim, hdim) # model to train
sgd = optim.SGD(model.parameters(), lr=0.001, momentum=0.9) # SGD 

MAX_ITER = 10

for i in range(MAX_ITER): 
    
    tic = time.time()
    sgd.zero_grad()
    loglik = mle(seq, model) # compute log-likelihood
    loss = -loglik 
    loss.backward() # compute gradient
    sgd.step()
    toc = time.time()
    
    print(f"Iter-{i}: log-likelihood = {float(loglik):.4f}; time cost = {toc-tic:.4f}")

Iter-0: log-likelihood = -2242.1567; time cost = 0.6457
Iter-1: log-likelihood = -2153.4778; time cost = 0.6506
Iter-2: log-likelihood = -1988.6143; time cost = 0.7776
Iter-3: log-likelihood = -1753.1870; time cost = 0.8019
Iter-4: log-likelihood = -1449.4432; time cost = 0.8723
Iter-5: log-likelihood = -1103.1483; time cost = 0.8863
Iter-6: log-likelihood = -784.9104; time cost = 0.8146
Iter-7: log-likelihood = -568.1177; time cost = 0.8153
Iter-8: log-likelihood = -455.4929; time cost = 0.8165
Iter-9: log-likelihood = -412.7558; time cost = 0.8815


### Predict next event time and type by sampling (approx. MBR)

In [104]:
def predict_time(model): 
    dts, ks = [], []
    n = 10 
    k, dt = thinning(model)
    dts += [float(dt)]
    dt_pred = np.mean(dts)
    return dt_pred

def predict_type(model, dt): 
    intens = model.compute_intensities(dt)
    k_pred = torch.argmax(intens)
    return k_pred

In [105]:
se = 0.0
nerr = 0

model.start() # restart
n = 100

for i, s in enumerate(seq[:n]): 
    # predict
    dt_pred = predict_time(model)
    # time
    dt = seq[i][0]
    se += (dt_pred - dt) ** 2
    # type 
    k_pred = predict_type(model, dt)
    k = seq[i][1]
    if k_pred != k: 
        nerr += 1

print(f"check time prediction accuracy")
print(f"RMSE using trained model : {np.sqrt(se/n):.4f}")

print(f"\ncheck type prediction error rate")
print(f"Error Rate using true model : {100.0*nerr/n:.2f}%")
print(f"Error Rate of random guess : {100.0*(1-1/kdim):.2f}%")


check time prediction accuracy
RMSE using trained model : 1.4219

check type prediction error rate
Error Rate using true model : 85.00%
Error Rate of random guess : 96.88%


### Train NHP by NCE

In [106]:
class SimpleNoise(nn.Module): 
    """
    a simple noise distribution -- Poisson process
    """
    
    def __init__(self, kdim, total_intensity): 
        super(SimpleNoise, self).__init__()
        """
        kdim : # of event types 
        """
        self.total_intensity = total_intensity
        self.inten = total_intensity / kdim
        self.intens = torch.zeros([kdim], dtype = torch.float32).fill_(self.inten)
        return 
    
    def start(self): 
        # do nothing
        return 
    
    def update(self, k, dt): 
        """
        k : event type 
        dt : elapsed time since last event
        """
        # does nothing
        # simple distribution, no dependence on history
        return 
    
    def forward(self, k, dt): 
        self.update(k, dt)
        return 
    
    def compute_intensities(self, dt): 
        return self.intens
    
    def compute_total_intensity(self, dt): 
        return self.total_intensity
    
    def draw(self, dt): 
        noise_events = [] # a collection of noise events over given interval
        dtj = 0.0
        while True: 
            # draw noise time (inversion sampling)
            u = np.random.uniform(0.0, 1.0)
            dtj += -np.log(1-u) / self.total_intensity
            
            if dtj <= dt: 
                # draw noise type
                kj = torch.multinomial(self.intens, 1)[0]
                noise_events += [(dtj, kj)]
            else: 
                break 
        
        return noise_events

In [99]:
def nce(data, model, noise): 
    """
    compute log-probability of correct discrimination under model and noise
    """
    model.start()
    noise.start()
    loglik = 0.0
    
    for dt, k in seq: 
        
        # real event & noise non-event
        p_real = model.compute_intensities(dt)[k]
        q_real = noise.compute_intensities(dt)[k]
        loglik += torch.log(p_real / (p_real + q_real))
        
        # real non-event & noise event
        for dtj, kj in noise.draw(dt): 
            p_noise = model.compute_intensities(dtj)[kj]
            q_noise = noise.compute_intensities(dtj)[kj]
            loglik += torch.log(q_noise / (p_noise + q_noise))
        
        # update model and noise with real event
        # both model and noise conditioned on real history
        model.forward(k, dt)
        noise.forward(k, dt)
    
    return loglik

In [107]:
model = NHP(kdim, hdim) # model to train
noise = SimpleNoise(kdim, len(seq)*1.0/T) # noise distribution q 

sgd = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

MAX_ITER = 50

for i in range(MAX_ITER): 
    
    tic = time.time()
    sgd.zero_grad()
    loglik = nce(seq, model, noise)
    loss = -loglik 
    loss.backward()
    sgd.step()
    toc = time.time()
    
    loglik = mle(seq, model)
    
    print(f"Iter-{i}: log-likelihood = {float(loglik):.4f}; time cost = {toc-tic:.4f}")

Iter-0: log-likelihood = -2330.4666; time cost = 0.2368
Iter-1: log-likelihood = -2320.1770; time cost = 0.2308
Iter-2: log-likelihood = -2306.1011; time cost = 0.2290
Iter-3: log-likelihood = -2288.8208; time cost = 0.2474
Iter-4: log-likelihood = -2268.6750; time cost = 0.2579
Iter-5: log-likelihood = -2246.9258; time cost = 0.2637
Iter-6: log-likelihood = -2222.8083; time cost = 0.2683
Iter-7: log-likelihood = -2196.7983; time cost = 0.2964
Iter-8: log-likelihood = -2167.4907; time cost = 0.2952
Iter-9: log-likelihood = -2137.0186; time cost = 0.2855
Iter-10: log-likelihood = -2104.3833; time cost = 0.3175
Iter-11: log-likelihood = -2068.4480; time cost = 0.3026
Iter-12: log-likelihood = -2030.2980; time cost = 0.3029
Iter-13: log-likelihood = -1990.9193; time cost = 0.3390
Iter-14: log-likelihood = -1950.4828; time cost = 0.3859
Iter-15: log-likelihood = -1908.9060; time cost = 0.3653
Iter-16: log-likelihood = -1866.5612; time cost = 0.3577
Iter-17: log-likelihood = -1821.6584; tim